# **Parallelism in Initial Design**
In `spotpython`, we provide a wrapper function, that encapsulates the objective function to enable its parallel execution via `multiprocessing` or `joblib`, allowing multiple configurations to be evaluated at the same time.

## Example
***
### Setup
To demonstrate the performance gain enabled by parallelization, we use a similar example to that in Section 47, where we perform hyperparameter tuning with `spotpython`and `PyTorch` Lightning on the Diabetes dataset using a ResNet model. We compare the time required with and without parallelization.

First, we import the necessary libraries, including the wrapper function `make_parallel`. We then define the `fun_control` and `design_control` settings. For `design_control`, we deliberately choose a larger initial design size of 80 in order to clearly demonstrate the performance gain.

In [ ]:
from spotpython.data.diabetes import Diabetes
from spotpython.hyperdict.light_hyper_dict import LightHyperDict
from spotpython.fun.hyperlight import HyperLight
from spotpython.utils.init import (fun_control_init, design_control_init)
from spotpython.hyperparameters.values import set_hyperparameter
from spotpython.spot import Spot
from math import inf
from spotpython.utils.parallel import make_parallel
import time

dataset = Diabetes()

fun_control = fun_control_init(
    fun_evals=80,
    max_time=inf,
    data_set=dataset,
    core_model_name="light.regression.NNResNetRegressor",
    hyperdict=LightHyperDict,
    _L_in=10,
    _L_out=1,
    seed=125,
    tensorboard_log=True,
    TENSORBOARD_CLEAN=True,
    
)

set_hyperparameter(fun_control, "optimizer", ["Adadelta", "Adam", "Adamax"])
set_hyperparameter(fun_control, "l1", [2, 5])
set_hyperparameter(fun_control, "epochs", [5, 8])
set_hyperparameter(fun_control, "batch_size", [5, 8])
set_hyperparameter(fun_control, "dropout_prob", [0.0, 0.5])
set_hyperparameter(fun_control, "patience", [2, 3])
set_hyperparameter(fun_control, "lr_mult", [0.1, 10.0])

design_control = design_control_init(
    init_size=80
)

fun = HyperLight().fun

We now measure the time required for sequential and parallel evaluation, beginning with the sequential approach.

In [ ]:
start1 = time.time()
spot_tuner = Spot(fun=fun, fun_control=fun_control, design_control=design_control)
res = spot_tuner.run()
end1 = time.time()

In [3]:
print(end1 - start1)

322.39894580841064


To use `make_parallel`, the number of cores must be specified via the `num_cores` parameter. By default, the function utilizes `multiprocessing`, but other parallelization methods can be selected using the `method` argument.

In [ ]:
start2 = time.time()
parallel_fun = make_parallel(fun, num_cores=8)
spot_parallel_tuner = Spot(fun=parallel_fun, fun_control=fun_control, design_control=design_control)
res = spot_parallel_tuner.run()
end2 = time.time()

In [7]:
print(end2 - start2)

96.39355731010437


### Results
As we can see, the sequential execution took 322 seconds, while the parallel execution with 8 cores and `multiprocessing` required only 96 seconds. This corresponds to an improvement of approximately **70%** on an initial design of 80.
## Notes
***
### OS
Linux uses the `fork` method by default to start new processes, whereas macOS and Windows use the `spawn` method. This leads to differences in how processes are handled across operating systems. We use the functionality of `set_all_seeds` to ensure that the evaluation remains reproducible across all operating systems.



##